1.  a) False

    b) False 
    
    c) False
    
    d) False
    
    e) False
    
    f) True
    
    g) False
    
    h) False
    
    i) True
    
    j) False
2. With larger datasets that takes models longer to run. GridSearch will try all possible parameter combinations making it take much longer. 

3. It works by using predictions from base learners as inputs and the actual responses as outputs to train a high-level or meta learner. 

4. Yes. by ensembling these models by either an average or an vote it would decrease the varianace in the models and in return boost the scores. 

5. n_estimators

6. a, support vector machine

7. Each estimator will be its own random forest tree system

8. b

9.  a) The top score on the test set would be with 800 trees, so that would be optimal

    b) There is probably an overfitting issue happening on the test data

10. Heterogeneous is an ensemble with different base learner algorithms, and homogeneous is when the same algorithms is used to ensemble. 

11. d

12. It is not a good idea. With cv set to 5, at least one fold will be trained on a subset of the data that does not have a fraudulent trasaction
    
13. a) max_depth

    b) learning rate

14. by building models with different numbers of trees and then choosing the model which scores the best

15. a) regression, the objective parameter is set to squared error which is a regression metric

    b) the max depth is set to 30 which is very high and likily to overfit the data 
    
16. n_estimators and max_depth. increasing them too much could casue overfitting but not having them large enough could underfit the model. 

In [ ]:
pip install optuna

In [ ]:
pip install xgboost

In [16]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import optuna

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import recall_score, classification_report, mean_squared_error
from sklearn.impute import SimpleImputer, KNNImputer
from scipy.stats import boxcox
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC, SVR
from precision_recall_cutoff import precision_recall_cutoff
from xgboost import XGBRegressor

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'Customers.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
customers = pd.read_csv(file_content_stream)
customers.head()

,CustomerID,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,1,Male,19,15000,39,Healthcare,1,4
1,2,Male,21,35000,81,Engineer,3,3
2,3,Female,20,86000,6,Engineer,1,1
3,4,Female,23,59000,77,Lawyer,0,2
4,5,Female,31,38000,40,Entertainment,2,6


In [17]:
# dropping id
customers = customers.drop(columns = 'CustomerID', axis = 0)
customers.head()

,Gender,Age,Annual Income ($),Spending Score (1-100),Profession,Work Experience,Family Size
0,Male,19,15000,39,Healthcare,1,4
1,Male,21,35000,81,Engineer,3,3
2,Female,20,86000,6,Engineer,1,1
3,Female,23,59000,77,Lawyer,0,2
4,Female,31,38000,40,Entertainment,2,6


In [18]:
# changing Profession and Gender
customers['Gender'] = np.where(customers['Gender'] == 'Male', 1, 0)
customers = pd.concat([customers.drop(columns = 'Profession', axis =  1 ), pd.get_dummies(customers['Profession'])], axis = 1)
customers.head()

,Gender,Age,Annual Income ($),Spending Score (1-100),Work Experience,Family Size,Artist,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,Marketing
0,1,19,15000,39,1,4,0,0,0,0,0,1,0,0,0
1,1,21,35000,81,3,3,0,0,1,0,0,0,0,0,0
2,0,20,86000,6,1,1,0,0,1,0,0,0,0,0,0
3,0,23,59000,77,0,2,0,0,0,0,0,0,0,1,0
4,0,31,38000,40,2,6,0,0,0,1,0,0,0,0,0


In [19]:
customers['heredity_1'] = customers['Annual Income ($)'] * customers['Age']
customers['heredity_2'] = customers['Annual Income ($)'] * customers['Work Experience']
customers['DT_2'] = np.where((customers['Age'] <= 32.5) & (customers['Annual Income ($)'] <= 149099), 1, 0)

In [20]:
# defining imput and target
x = customers[['Annual Income ($)', 'heredity_1', 'heredity_2', 'Age', 'DT_2', 'Work Experience', 'Family Size']]
y = customers['Spending Score (1-100)']

# splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [25]:
class objective:
    
    def __init__(self, seed):
        self.seed = seed
        
    def __call__(self, trial):
        params = dict(n_estimators = trial.suggest_int('n_estimators', 100, 1000),
                     max_depth = trial.suggest_int('max_depth', 2, 10),
                     min_samples_split = trial.suggest_int('min_samples_split', 3, 10),
                     min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10))
        
        scores = list()
        
        skf = KFold(n_splits = 3, shuffle = True, random_state = self.seed)
        
        for train_idx, test_idx in skf.split(x, y):
            
            ## Splitting the data 
            x_train, x_test = x.iloc[train_idx], x.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            rf_md = RandomForestRegressor(**params).fit(x_train, y_train)
            
            rf_pred = rf_md.predict(x_test)
            
            scores.append(mean_squared_error(y_test, rf_pred, squared = False))
        
        return np.mean(scores)

seed = 42
n_trials = 50

study_meta = optuna.create_study(direction = 'maximize')
study_meta.optimize(objective(seed), n_trials = n_trials)
study_meta.best_trial.params

# running model with best parameters
rf_opt = RandomForestRegressor(**study_meta.best_trial.params).fit(x_train, y_train)

# predicting on test
rf_pred = rf_opt.predict(x_test)

rf_results = mean_squared_error(y_test, rf_pred, squared = False)
print('the RMSE of the rf model is: ', rf_results)

[I 2023-04-02 21:23:48,923] A new study created in memory with name: no-name-8b0b2286-b415-4388-ad3f-0e00cde6553a
[I 2023-04-02 21:23:51,847] Trial 0 finished with value: 27.945783116040573 and parameters: {'n_estimators': 394, 'max_depth': 4, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 0 with value: 27.945783116040573.
[I 2023-04-02 21:23:55,050] Trial 1 finished with value: 28.295122183972524 and parameters: {'n_estimators': 312, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 7}. Best is trial 1 with value: 28.295122183972524.
[I 2023-04-02 21:23:57,813] Trial 2 finished with value: 28.47199415740182 and parameters: {'n_estimators': 251, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 8}. Best is trial 2 with value: 28.47199415740182.
[I 2023-04-02 21:24:01,055] Trial 3 finished with value: 28.196501687787816 and parameters: {'n_estimators': 347, 'max_depth': 7, 'min_samples_split': 10, 'min_samples_leaf': 10}. Best is trial 2 with value: 2

the RMSE of the rf model is:  28.687012638454767


In [26]:
class objective:
    
    def __init__(self, seed):
        self.seed = seed
        
    def __call__(self, trial):
        params = dict(n_estimators = trial.suggest_int('n_estimators', 100, 1000),
                     max_depth = trial.suggest_int('max_depth', 2, 10),
                     min_samples_split = trial.suggest_int('min_samples_split', 3, 10),
                     min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10),
                     learning_rate = trial.suggest_float('learning_rate', 0.01, .1))
        
        scores = list()
        
        skf = KFold(n_splits = 3, shuffle = True, random_state = self.seed)
        
        for train_idx, test_idx in skf.split(x, y):
            
            ## Splitting the data 
            x_train, x_test = x.iloc[train_idx], x.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            gb_md = GradientBoostingRegressor(**params).fit(x_train, y_train)
            
            gb_pred = gb_md.predict(x_test)
            
            scores.append(mean_squared_error(y_test, gb_pred, squared = False))
        
        return np.mean(scores)

seed = 42
n_trials = 50

study_meta = optuna.create_study(direction = 'maximize')
study_meta.optimize(objective(seed), n_trials = n_trials)
study_meta.best_trial.params

# running model with best parameters
gb_opt = GradientBoostingRegressor(**study_meta.best_trial.params).fit(x_train, y_train)

# predicting on test
gb_pred = gb_opt.predict(x_test)

gb_results = mean_squared_error(y_test, gb_pred, squared = False)
print('the RMSE of the gb model is: ', gb_results)

[I 2023-04-02 21:26:27,385] A new study created in memory with name: no-name-07aef242-a366-4d6e-aa9c-31bf9c282302
[I 2023-04-02 21:26:32,951] Trial 0 finished with value: 32.29133829282561 and parameters: {'n_estimators': 407, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 4, 'learning_rate': 0.0940605874782595}. Best is trial 0 with value: 32.29133829282561.
[I 2023-04-02 21:26:36,819] Trial 1 finished with value: 31.534311395376722 and parameters: {'n_estimators': 286, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 6, 'learning_rate': 0.08118210489049679}. Best is trial 0 with value: 32.29133829282561.
[I 2023-04-02 21:26:44,048] Trial 2 finished with value: 30.871926664933284 and parameters: {'n_estimators': 779, 'max_depth': 5, 'min_samples_split': 7, 'min_samples_leaf': 10, 'learning_rate': 0.03757983114066736}. Best is trial 0 with value: 32.29133829282561.
[I 2023-04-02 21:26:50,662] Trial 3 finished with value: 30.1246100333526 and parameters: {'n_esti

the RMSE of the gb model is:  32.5361648933108


In [27]:
class objective:
    
    def __init__(self, seed):
        self.seed = seed
        
    def __call__(self, trial):
        params = dict(n_estimators = trial.suggest_int('n_estimators', 300, 500),
                     max_depth = trial.suggest_int('max_depth', 3, 7),
                     min_child_weight = trial.suggest_int('min_child_weight', 5, 7),
                     learning_rate= trial.suggest_float('learning_rate', 0.01, 0.1),
                     gamma= trial.suggest_float('gamma', 0.1, 0.3),
                     subsample= trial.suggest_float('subsample', 0.8, 1))
        
        scores = list()
        
        skf = KFold(n_splits = 3, shuffle = True, random_state = self.seed)
        
        for train_idx, test_idx in skf.split(x, y):
            
            ## Splitting the data 
            x_train, x_test = x.iloc[train_idx], x.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            xgb_md = XGBRegressor(**params).fit(x_train, y_train)
            
            xgb_pred = xgb_md.predict(x_test)
            
            scores.append(mean_squared_error(y_test, xgb_pred, squared = False))
        
        return np.mean(scores)

seed = 42
n_trials = 50

study_meta = optuna.create_study(direction = 'maximize')
study_meta.optimize(objective(seed), n_trials = n_trials)
study_meta.best_trial.params

# running model with best parameters
xgb_opt = XGBRegressor(**study_meta.best_trial.params).fit(x_train, y_train)

# predicting on test
xgb_pred = xgb_opt.predict(x_test)

xgb_results = mean_squared_error(y_test, xgb_pred, squared = False)
print('the RMSE of the xgb model is: ', xgb_results)

[I 2023-04-02 21:34:37,049] A new study created in memory with name: no-name-4416ddd6-7f5a-4020-b3f0-499bde6ab6c3
[I 2023-04-02 21:34:39,667] Trial 0 finished with value: 29.313781203656177 and parameters: {'n_estimators': 487, 'max_depth': 5, 'min_child_weight': 7, 'learning_rate': 0.019167503530526278, 'gamma': 0.16180525975393595, 'subsample': 0.8253374834414294}. Best is trial 0 with value: 29.313781203656177.
[I 2023-04-02 21:34:40,214] Trial 1 finished with value: 29.491748003714644 and parameters: {'n_estimators': 314, 'max_depth': 3, 'min_child_weight': 7, 'learning_rate': 0.08024474224071851, 'gamma': 0.2363534363079494, 'subsample': 0.9048309089958209}. Best is trial 1 with value: 29.491748003714644.
[I 2023-04-02 21:34:41,201] Trial 2 finished with value: 30.861605420628972 and parameters: {'n_estimators': 495, 'max_depth': 4, 'min_child_weight': 7, 'learning_rate': 0.06939405377119699, 'gamma': 0.13785050812003813, 'subsample': 0.819827126301777}. Best is trial 2 with value

the RMSE of the xgb model is:  32.13531717829318


In [28]:
print('Random Forest RMSE: ', rf_results)
print('Gradient Boosting RMSE: ', gb_results)
print('XGBoost RMSE: ', xgb_results)

Random Forest RMSE:  28.687012638454767
Gradient Boosting RMSE:  32.5361648933108
XGBoost RMSE:  32.13531717829318


In [31]:
ens_preds = (rf_pred + gb_pred + xgb_pred) / 3
print('the RMSE of the xgb model is: ', mean_squared_error(y_test, ens_preds, squared = False))

the RMSE of the xgb model is:  30.450765389712373


In [33]:
# based on these results the random forest model performed the best because it had the lowest RMSE. therefore, i would choose that model to predictin predicting spending score